In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,34835
2,Huelva,Confirmados PDIA 14 días,1239
3,Huelva,Tasa PDIA 14 días,"241,4404583276497"
4,Huelva,Confirmados PDIA 7 días,685
5,Huelva,Total Confirmados,35024
6,Huelva,Curados,32633
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  34835.0
/tmp/ipykernel_119245/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9563.0
/tmp/ipykernel_119245/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_119245/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_119245/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_119245/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 612 personas en los últimos 7 días 

Un positivo PCR cada 395 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,34835.0,685.0,1239.0,513170.0,133.484031,241.440458,159.0
Huelva-Costa,20222.0,479.0,846.0,289548.0,165.430257,292.179535,102.0
Huelva (capital),9563.0,235.0,364.0,143837.0,163.379381,253.064232,55.0
Condado-Campiña,11231.0,162.0,318.0,156231.0,103.692609,203.544751,47.0
Moguer,1483.0,40.0,58.0,21867.0,182.924041,265.239859,19.0
Isla Cristina,2379.0,79.0,171.0,21393.0,369.279671,799.326883,13.0
Sierra de Huelva-Andévalo Central,3011.0,31.0,57.0,67391.0,46.000208,84.581027,10.0
Cartaya,1451.0,22.0,34.0,20083.0,109.545387,169.297416,9.0
Punta Umbría,884.0,14.0,22.0,15355.0,91.175513,143.275806,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,35.0,3.0,18.0,1043.0,287.631831,1725.790988,0.0
Niebla,303.0,24.0,54.0,4116.0,583.090379,1311.953353,6.0
Isla Cristina,2379.0,79.0,171.0,21393.0,369.279671,799.326883,13.0
Zarza-Perrunal (La),33.0,3.0,6.0,1253.0,239.425379,478.850758,0.0
Alosno,208.0,1.0,16.0,3933.0,25.425884,406.814137,1.0
Lucena del Puerto,280.0,4.0,12.0,3261.0,122.661760,367.985281,1.0
Lepe,2286.0,55.0,95.0,27880.0,197.274032,340.746055,3.0
Ayamonte,1438.0,36.0,67.0,21104.0,170.583776,317.475360,6.0
Huelva-Costa,20222.0,479.0,846.0,289548.0,165.430257,292.179535,102.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,208.0,1.0,16.0,3933.0,25.425884,406.814137,1.0,0.062500
Trigueros,345.0,1.0,9.0,7862.0,12.719410,114.474688,1.0,0.111111
Santa Bárbara de Casa,35.0,3.0,18.0,1043.0,287.631831,1725.790988,0.0,0.166667
Almonte,1958.0,11.0,36.0,24507.0,44.885135,146.896805,2.0,0.305556
Lucena del Puerto,280.0,4.0,12.0,3261.0,122.661760,367.985281,1.0,0.333333
Cerro de Andévalo (El),162.0,2.0,5.0,2327.0,85.947572,214.868930,0.0,0.400000
San Bartolomé de la Torre,313.0,2.0,5.0,3761.0,53.177346,132.943366,0.0,0.400000
Bollullos Par del Condado,1062.0,7.0,17.0,14387.0,48.655036,118.162230,2.0,0.411765
Valverde del Camino,683.0,5.0,12.0,12750.0,39.215686,94.117647,2.0,0.416667
